In [16]:
import gzip 
from lxml import etree
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb



In [17]:
DOSSIER_500 = "./todo/test1000_2/"
DOSSIER_TODO = './todo/'

In [18]:
def timtim(fonction):
    def wrapper(*args, **kwargs):
        debut = time.time()
        resultat = fonction(*args, **kwargs)
        fin = time.time()
        temps_execution = fin - debut
        print(f"La fonction {fonction.__name__} a pris {temps_execution:.5f} secondes pour s'exécuter.")
        return resultat
    return wrapper

In [19]:
def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

In [20]:
@timtim
def Methode_1(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   liste_methode_1 = []
   for ligne_methode_1 in docbase:
     dict_temp = {}
     for clef, valolo in ligne_methode_1.items():
         if isinstance(valolo, dict) and '@V' in valolo:
             val_atrib = valolo['@V']
         else:
             val_atrib = valolo
         dict_temp[clef] = val_atrib
     liste_methode_1.append(dict_temp)
   df_methode_1 = pd.DataFrame(liste_methode_1)
   liste_df.append(df_methode_1)
  except Exception as e :
   print(f'Erreur fichier {fichier}') 
   continue 
 
 df_gros_1 = pd.concat(liste_df)
 return df_gros_1

@timtim
def Methode_2(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   df_methode_2 = pd.DataFrame(docbase)
   nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
   for col in df_methode_2.columns : 
    df_methode_2[col] = df_methode_2[col].apply(nettoyage)
   liste_df.append(df_methode_2)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
   
 gros_df_2 = pd.concat(liste_df)
 return gros_df_2


@timtim
def Methode_3(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   df_methode_3 = pd.json_normalize(docbase)
   liste_df.append(df_methode_3)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
 gros_df_3 = pd.concat(liste_df)
 return gros_df_3

SCHEMA = {"Nature" : pl.Object,
          "LibCpte" : pl.Object,
          "Fonction" : pl.Object,
          "Operation" : pl.Object,
          "ContNat" : pl.Object,
          "ArtSpe" : pl.Object,
          "ContFon" : pl.Object,
          "ContOp" : pl.Object,
          "CodRD" : pl.Object,
          "MtBudgPrec" : pl.Object,
          "MtRARPrec" : pl.Object,
          "MtPropNouv" : pl.Object,
          "MtPrev" : pl.Object,
          "CredOuv" : pl.Object,
          "MtReal" : pl.Object,
          "MtRAR3112" : pl.Object,
          "OpBudg" : pl.Object,
          "TypOpBudg" : pl.Object,
          "OpeCpteTiers" : pl.Object, 
          "MtSup" : pl.Object, 
          "CaracSup" : pl.Object
          }

@timtim
def Methode_polars(chemin, schema) : 
 liste_prepl = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   pre_df = pl.DataFrame(docbase, schema=schema)
   liste_prepl.append(pre_df)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
 pl_test = pl.concat(liste_prepl)
 return pl_test 

In [22]:
dfm1 = Methode_1(DOSSIER_TODO)

Erreur fichier ./todo\619027.xml.gz
Erreur fichier ./todo\628857.xml.gz
Erreur fichier ./todo\682109.xml.gz
Erreur fichier ./todo\722795.xml.gz
Erreur fichier ./todo\730073.xml.gz
Erreur fichier ./todo\748147.xml.gz
Erreur fichier ./todo\756300.xml.gz
Erreur fichier ./todo\775775.xml.gz
Erreur fichier ./todo\807134.xml.gz
Erreur fichier ./todo\809162.xml.gz
Erreur fichier ./todo\809290.xml.gz
Erreur fichier ./todo\819789.xml.gz
Erreur fichier ./todo\821461.xml.gz
Erreur fichier ./todo\838663.xml.gz
La fonction Methode_1 a pris 341.08701 secondes pour s'exécuter.


In [23]:
dfm2 = Methode_2(DOSSIER_TODO)

Erreur fichier ./todo\775775.xml.gz
La fonction Methode_2 a pris 414.56070 secondes pour s'exécuter.


In [35]:
dfm2.columns

Index(['Nature', 'ContNat', 'ArtSpe', 'CodRD', 'MtBudgPrec', 'MtPropNouv',
       'MtPrev', 'CredOuv', 'OpBudg', 'MtSup', 'CaracSup', 'TypOpBudg',
       'MtRARPrec', 'LibCpte', 'Operation', 'ContOp', 'MtReal', 'MtRAR3112',
       'Fonction', 'OpeCpteTiers', 'ContFon'],
      dtype='object')

In [24]:
dfm3 = Methode_3(DOSSIER_TODO)

Erreur fichier ./todo\775775.xml.gz
La fonction Methode_3 a pris 386.22054 secondes pour s'exécuter.


In [34]:
dfm3.columns

Index(['Nature.@V', 'ContNat.@V', 'ArtSpe.@V', 'CodRD.@V', 'MtBudgPrec.@V',
       'MtPropNouv.@V', 'MtPrev.@V', 'CredOuv.@V', 'OpBudg.@V', 'MtSup.@V',
       'MtSup.@Code', 'CaracSup.@V', 'CaracSup.@Code', 'TypOpBudg.@V', 'MtSup',
       'MtRARPrec.@V', 'CaracSup', 'LibCpte.@V', 'Operation.@V', 'ContOp.@V',
       'MtReal.@V', 'MtRAR3112.@V', 'Fonction.@V', 'OpeCpteTiers.@V',
       'ContFon.@V'],
      dtype='object')

In [ ]:
comparatif_pl = Methode_polars(DOSSIER_TODO, schema=SCHEMA)

In [36]:
comparatif_pl.columns

['Nature',
 'LibCpte',
 'Fonction',
 'Operation',
 'ContNat',
 'ArtSpe',
 'ContFon',
 'ContOp',
 'CodRD',
 'MtBudgPrec',
 'MtRARPrec',
 'MtPropNouv',
 'MtPrev',
 'CredOuv',
 'MtReal',
 'MtRAR3112',
 'OpBudg',
 'TypOpBudg',
 'OpeCpteTiers',
 'MtSup',
 'CaracSup']

Outpout : 

**Methode_1**, extraction depuis les dictionnaires : 

- Erreur fichier ./todo\619027.xml.gz
- Erreur fichier ./todo\628857.xml.gz
- Erreur fichier ./todo\682109.xml.gz
- Erreur fichier ./todo\722795.xml.gz
- Erreur fichier ./todo\730073.xml.gz
- Erreur fichier ./todo\748147.xml.gz
- Erreur fichier ./todo\756300.xml.gz
- Erreur fichier ./todo\775775.xml.gz
- Erreur fichier ./todo\807134.xml.gz
- Erreur fichier ./todo\809162.xml.gz
- Erreur fichier ./todo\809290.xml.gz
- Erreur fichier ./todo\819789.xml.gz
- Erreur fichier ./todo\821461.xml.gz
- Erreur fichier ./todo\838663.xml.gz

- La fonction Methode_1 a pris **341.08701** secondes pour s'exécuter.



**Methode_2**, DataFrame : 

- Erreur fichier ./todo\775775.xml.gz

- La fonction Methode_2 a pris **414.56070** secondes pour s'exécuter.


**Methode_3**, json_normalize : 

- Erreur fichier ./todo\775775.xml.gz

- La fonction Methode_3 a pris **386.22054** secondes pour s'exécuter.


**Methode Polars** : 

- Erreur fichier ./todo\619027.xml.gz
- Erreur fichier ./todo\628857.xml.gz
- Erreur fichier ./todo\682109.xml.gz
- Erreur fichier ./todo\722795.xml.gz
- Erreur fichier ./todo\730073.xml.gz
- Erreur fichier ./todo\748147.xml.gz
- Erreur fichier ./todo\756300.xml.gz
- Erreur fichier ./todo\775775.xml.gz
- Erreur fichier ./todo\807134.xml.gz
- Erreur fichier ./todo\809162.xml.gz
- Erreur fichier ./todo\809290.xml.gz
- Erreur fichier ./todo\819789.xml.gz
- Erreur fichier ./todo\821461.xml.gz
- Erreur fichier ./todo\838663.xml.gz

- La fonction Methode_polars a pris **366.51400** secondes pour s'exécuter.


### Resultats sur 20 000 fichiers, pour l'extract des lignes budgetaires : 

- Methode extract depuis dict : 341.08701 sec, 14 erreurs 
- Methode DataFrame : 414.56070 sec, 1 erreur
- Methode json : 386.22054 sec, 1 erreur
- Methode Polars : 366.51400 sec, 14 erreurs 

Une erreure commune parmi les 4 méthode, sur le fichier 775775, il s'avère qu'il est impossible à ouvrir car l'archive est endommagée. 

La méthode Json est plus rapide sur le traitement, notamment car elle n'a pas besoin de nettoyer les @V dans les données, les @V n'apparaissent que sur les libelles des colonnes, cependant la méthode json_normalize traite les Carac Sup et Mt Sup de façon particulière, les rendants inexploitables. Ce gain de temps sera donc perdu lorsque l'on voudra utiliser ces informations. 

La méthode DataFrame elle, considère Mtsup et CaracSup comme des colonnes composées de listes de dictionnaires / de dictionnaires, les rendant exploitables plus tard. 

### Ajout de la verif sur le fichier parquet 

1. Methode Duckdb 

In [ ]:
from minio import Minio
import duckdb

client = Minio(
    endpoint='localhost:9000',
    access_key='airflow-local',
    secret_key='airflow-local',
    secure = False
  )

In [3]:
def recherche_id_fichier() :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet('./parquet-proto-doc-budget')
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id

In [26]:
liste_ttt = []
for i in glob.glob(os.path.join(DOSSIER_TODO, "*.gz")) : 
 liste_ttt.append(i)

liste_ttt[7]

'../../todo\\20201113-1875908.xml.gz'

In [28]:
fichier_test = '../../todo/727285.xml.gz'
id_test = _isolement_id(liste_ttt[1])
id_test

'1825601'

In [32]:
dict_meta = test_parse['DocumentBudgetaire']['EnTeteDocBudgetaire']
dict_meta

{'DteStr': {'@V': '2021-06-02'},
 'LibellePoste': {'@V': 'le Trésorier de la MEL'},
 'IdPost': {'@V': '059053'},
 'LibelleColl': {'@V': 'METROPOLE EUROPEENNE DE LILLE'},
 'IdColl': {'@V': '20009320100016'},
 'NatCEPL': {'@V': '06 Groupements de collectivités'}}

In [ ]:
dict_meta['Id_Fichier'] = {'@V' : id_test}
dict_meta

dict_glob = dict_parse
for i in dict_glob :
    i.update(dict_meta)
dict_glob

In [37]:
def recherche_id_fichier(chemin_parquet) :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id


def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité'''
 val_id_fichier_source = fichier.split("\\")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier

In [50]:
@timtim
def Methode_2_1(chemin, chemin_verif) : 
 liste_id = recherche_id_fichier(chemin_verif)
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  id_fichier = _isolement_id(fichier)
  #Fichier non pre existant dans la db / parquet
  if id_fichier in liste_id :
   pass 
  else : 
   try : 
     fichier_parse = parse_fichier(fichier)
     metadonnees = fichier_parse['DocumentBudgetaire']['EnTeteDocBudgetaire']
     donnes_budget = fichier_parse['DocumentBudgetaire']['Budget']['LigneBudget']
     metadonnees['Id_Fichier'] = {'@V' : id_fichier}
     for i in donnes_budget : 
      i.update(metadonnees)
     def_2_1 = pd.DataFrame(donnes_budget)
     nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
     for col in def_2_1.columns : 
      def_2_1[col] = def_2_1[col].apply(nettoyage)
     liste_df.append(def_2_1)
   except Exception as e : 
     print(f'Erreur fichier {fichier}')
     continue
 df_2_1 = pd.concat(liste_df)
 df_2_1.to_parquet('parquet_renouveau', compression= 'gzip')
 return df_2_1 


In [52]:
chemin_origin_parquet = './parquet-proto-doc-budget'
chemin_parquet_verif = './parquet_renouveau'
